In [18]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import nltk
import re
from nltk.corpus import stopwords
from sklearn.metrics import mean_squared_error as mse


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Edward\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Edward\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Edward\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Edward\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [7]:
train_df.columns

Index(['id', 'url_legal', 'license', 'excerpt', 'target', 'standard_error',
       'processed_text'],
      dtype='object')

In [3]:
def preprocess_text(text_data, type='train'):
    text_processed = []
    for e in text_data['excerpt']:
        # find alphabets
        e = re.sub("[^a-zA-Z]", " ", e)

        # convert to lower case
        e = e.lower()

        # tokenize words
        e = nltk.word_tokenize(e)

        # remove stopwords
        e = [word for word in e if not word in set(stopwords.words("english"))]

        # lemmatization
        lemma = nltk.WordNetLemmatizer()
        e = [lemma.lemmatize(word) for word in e]
        e = " ".join(e)

        text_processed.append(e)
    return text_processed

In [5]:
train_df['processed_text'] = preprocess_text(train_df)
test_df['processed_text'] = preprocess_text(test_df)

In [34]:
vectorizer =  TfidfVectorizer(binary=True, ngram_range=(1, 1))
X_train = vectorizer.fit_transform(train_df['processed_text'])
X_test = vectorizer.transform(test_df['processed_text'])
y_train = train_df['target']

In [35]:
X_train, X_cv, y_train, y_cv = train_test_split(
    X_train, y_train, test_size=0.30, random_state=42)

In [36]:
X_train = X_train.toarray()
X_cv = X_cv.toarray()
X_test = X_test.toarray()

In [59]:
lambda_ = 0.005
model = Sequential([
    Dense(25, activation= "relu"),
    Dense(10, activation= 'relu'),
    Dense(5, activation='relu'),
    Dense(1)
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
             loss=tf.keras.losses.MeanSquaredError())

In [60]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
62/62 [==============================] - 2s 13ms/step - loss: 0.8443
Epoch 2/100
62/62 [==============================] - 1s 14ms/step - loss: 0.2736
Epoch 3/100
62/62 [==============================] - 1s 14ms/step - loss: 0.1226
Epoch 4/100
62/62 [==============================] - 1s 13ms/step - loss: 0.0806
Epoch 5/100
62/62 [==============================] - 1s 13ms/step - loss: 0.0669
Epoch 6/100
62/62 [==============================] - 1s 14ms/step - loss: 0.0590
Epoch 7/100
62/62 [==============================] - 1s 14ms/step - loss: 0.0554
Epoch 8/100
62/62 [==============================] - 1s 14ms/step - loss: 0.0499
Epoch 9/100
62/62 [==============================] - 1s 14ms/step - loss: 0.0431
Epoch 10/100
62/62 [==============================] - 1s 14ms/step - loss: 0.0380
Epoch 11/100
62/62 [==============================] - 1s 14ms/step - loss: 0.0298
Epoch 12/100
62/62 [==============================] - 1s 14ms/step - loss: 0.0271
Epoch 13/100
62/62 [=====

In [62]:
print(f'RMSE error: {mse(y_cv, model.predict(X_cv),)}')

27/27 [==============================] - 0s 7ms/step
RMSE error: 0.5830929914421591


In [63]:
import os
model.save(f'{os.path.join(os.getcwd(), "my_model.save")}')

INFO:tensorflow:Assets written to: D:\Projector\my_model.save\assets
